In [16]:
from pathlib import Path
import torch
import matplotlib.pyplot as plt
import torch

from torch import nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torchvision import transforms

from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import inception_v3, Inception_V3_Weights

from torchvision import datasets

from torch.utils.data import DataLoader

from torch import nn
from timeit import default_timer as timer

from going_modular.going_modular import engine

In [17]:
torch.cuda.empty_cache()
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = torch.device("cuda:0")

In [18]:
weights_eff = EfficientNet_V2_S_Weights.DEFAULT
model_eff = efficientnet_v2_s(weights=weights_eff).to(device)

In [19]:
weights_res = EfficientNet_V2_S_Weights.DEFAULT
model_res = efficientnet_v2_s(weights_eff).to(device)

b:\Softwares\anaconda3\envs\GG_2527\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [20]:
model_eff.classifier = torch.nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=5, bias=True),
    ).to(device)

model_res.classifier = torch.nn.Sequential(
    nn.Linear(1280, 256), # Additional linear layer with 256 output features
    # nn.ReLU(inplace=True),         # Activation function
    nn.Dropout(p=0.5, inplace=True),
    nn.Linear(256, 5)
).to(device)

model_eff.load_state_dict(torch.load('weights/weights.tf'))
model_res.load_state_dict(torch.load('weights/w1.tf'))

<All keys matched successfully>

In [21]:
data_path = Path("data/")
image_path = data_path / "events"
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir
output_dir = 'output'

In [22]:
import numpy as np

start_time = timer()

# Setup training and save the results
class EnsembleModel(nn.Module):   
    def __init__(self, modelA, modelB, num_classes):
        super().__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.classifier = nn.Linear(1280, num_classes)  # Adjust the output size to match the number of classes
        
    def forward(self, x):
        x1 = self.modelA(x)
        x2 = self.modelB(x)
        x = torch.cat((x1, x2), dim=1)
        out = self.classifier(x)
        return out
    
ensemble_model = EnsembleModel(model_eff, model_res, num_classes=5)


for param in ensemble_model.parameters():
    param.requires_grad = False

for param in ensemble_model.classifier.parameters():
    param.requires_grad = True    

ensemble_model = ensemble_model.to(device)

In [23]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(224),
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomVerticalFlip(),
    # transforms.Resize(112),  # Downscale
    # transforms.Resize(224),
    # transforms.RandomRotation(20),
    # transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    # transforms.CenterCrop(224), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [24]:

        
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    transforms.RandomErasing(p=0.5, scale=(0.06, 0.08), ratio=(1, 3), value=0, inplace=True)
])

transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262])
])

In [25]:
import os
from sklearn.preprocessing import LabelEncoder
labels = os.listdir(train_dir)
labels
encoder_labels = LabelEncoder()
encoder_labels.fit(labels)
encoder_labels

LabelEncoder()

In [26]:
from sympy import root


train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=transforms_train, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=transforms_val)

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

In [27]:
# Get the length of class_names (one output unit for each class)
output_shape = len(labels)
# Recreate the classifier layer and seed it to the target device

ensemble_model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.1, inplace=True),
    nn.Linear(in_features=1280, out_features=output_shape, bias=True),
).to(device)

In [28]:
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1) # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.SGD(ensemble_model.parameters(), lr=0.03, momentum=0.9, weight_decay=0.01)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
# warmup_scheduler = WarmupLR(scheduler, warmup_factor=0.1, warmup_iters=10, warmup_method="linear")


In [29]:
import tqdm
import copy

import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
def training(model, model_name, num_epochs, train_dataloader, val_dataloader):

    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.33)

    train_loss_array = []
    train_acc_array = []
    val_loss_array = []
    val_acc_array = []
    lowest_val_loss = np.inf
    best_model = None

    for epoch in tqdm(range(num_epochs)):

        print('Epoch: {} | Learning rate: {}'.format(epoch + 1, scheduler.get_lr()))

        for phase in ['train', 'val']:

            epoch_loss = 0
            epoch_correct_items = 0
            epoch_items = 0

            if phase == 'train':
                model.train()
                with torch.enable_grad():
                    for samples, targets in train_dataloader:
                        samples = samples.to(device)
                        targets = targets.to(device)

                        optimizer.zero_grad()
                        outputs = model(samples)
                        loss = loss_function(outputs, targets)
                        preds = outputs.argmax(dim=1)
                        correct_items = (preds == targets).float().sum()
                        
                        loss.backward()
                        optimizer.step()

                        epoch_loss += loss.item()
                        epoch_correct_items += correct_items.item()
                        epoch_items += len(targets)

                train_loss_array.append(epoch_loss / epoch_items)
                train_acc_array.append(epoch_correct_items / epoch_items)

                scheduler.step()

            elif phase == 'val':
                model.eval()
                with torch.no_grad():
                    for samples, targets in val_dataloader:
                        samples = samples.to(device)
                        targets = targets.to(device)

                        outputs = model(samples)
                        loss = loss_function(outputs, targets)
                        preds = outputs.argmax(dim=1)
                        correct_items = (preds == targets).float().sum()

                        epoch_loss += loss.item()
                        epoch_correct_items += correct_items.item()
                        epoch_items += len(targets)

                val_loss_array.append(epoch_loss / epoch_items)
                val_acc_array.append(epoch_correct_items / epoch_items)

                if epoch_loss / epoch_items < lowest_val_loss:
                    lowest_val_loss = epoch_loss / epoch_items
                    torch.save(model.state_dict(), '{}_weights.pth'.format(model_name))
                    best_model = copy.deepcopy(model)
                    print("\t| New lowest val loss for {}: {}".format(model_name, lowest_val_loss))

    return best_model, train_loss_array, train_acc_array, val_loss_array, val_acc_array

In [30]:
import numpy as np

start_time = timer()

# Setup training and save the results
class EnsembleModel(nn.Module):   
    def __init__(self, modelA, modelB, num_classes):
        super().__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.classifier = nn.Linear(1280, num_classes)  # Adjust the output size to match the number of classes
        
    def forward(self, x):
        x1 = self.modelA(x)
        x2 = self.modelB(x)
        x = torch.cat((x1, x2), dim=1)
        out = self.classifier(x)
        return out


  0%|          | 0/50 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x10 and 1280x5)